In [2]:
import pandas as pd
import json
import numpy as np
from unidecode import unidecode
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [3]:
with open('data.json', encoding="utf8") as json_file:
    data = json.load(json_file)

df = pd.DataFrame.from_dict(data)
df.columns = ['token', 'attributes', 'description']
df = pd.concat([df.drop(['attributes'], axis=1), df['attributes'].apply(pd.Series)], axis=1).drop_duplicates()
df.head()

,token,description,کارکرد,مدل (سال تولید),رنگ,برند و تیپ,وضعیت موتور,وضعیت شاسی‌ها,وضعیت بدنه,مهلت بیمهٔ شخص ثالث,...,نوع سوخت,قیمت فروش نقدی,وضعیت شاسی جلو,وضعیت شاسی عقب,نمایشگاه,حداقل مبلغ پیش‌پرداخت,نحوهٔ فروش,تعداد اقساط,مبلغ قسط,فروشنده
0,wYtmuTDf,ساعات بازدید روزهای عادی 17.30 به بعد.,۲۴۲٬۰۰۰,۱۳۸۵,مشکی,پراید، صندوق‌دار، بنزینی,سالم,سالم و پلمپ,خط و خش جزیی,۱ ماه,...,بنزین,۹۳٬۰۰۰٬۰۰۰ تومان,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,wYO2A7PE,خودرو از لحاظ فنی بشرط کارشناسی بدنه چند لکه ...,۲۰۰,۲۰۱۴,نقره‌ای,بایک، سابرینا، هاچبک,سالم,سالم و پلمپ,خط و خش جزیی,۱۰ ماه,...,NaN,۳۲۵٬۰۰۰٬۰۰۰ تومان,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,wYti-uhK,با سلام\nفنی سالم.بی رنگ.معاوضه با خودرو ارزان...,۸۹٬۰۰۰,۱۳۹۴,سفید,آریو، اتوماتیک 1600cc,سالم,سالم و پلمپ,سالم و بی‌خط و خش,۶ ماه,...,بنزین,۴۳۰٬۰۰۰٬۰۰۰ تومان,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,wYteuShG,تمام سرویس ها تازه انجام شده ماشین کم کار و بس...,۴۸٬۰۰۰,۱۳۹۷,سفید,دانگ فنگ، H30 کراس,سالم,NaN,رنگ‌شدگی در ۱ ناحیه,۱۲ ماه,...,بنزین,۴۶۵٬۰۰۰٬۰۰۰ تومان,سالم و پلمپ,ضربه‌خورده,NaN,NaN,NaN,NaN,NaN,NaN
4,wYtW-Q6S,تاکسی بین شهری slx بسیار تروتمیز موتور سالم گی...,۶۰٬۰۰۰,۱۳۸۹,زرد,پژو، 405، SLX بنزینی,سالم,سالم و پلمپ,دوررنگ,۱۰ ماه,...,بنزین,۱۶۰٬۰۰۰٬۰۰۰ تومان,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15071 entries, 0 to 15766
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   token                  15071 non-null  object
 1   description            15071 non-null  object
 2   کارکرد                 15071 non-null  object
 3   مدل (سال تولید)        15071 non-null  object
 4   رنگ                    15071 non-null  object
 5   برند و تیپ             15071 non-null  object
 6   وضعیت موتور            14798 non-null  object
 7   وضعیت شاسی‌ها          13559 non-null  object
 8   وضعیت بدنه             15051 non-null  object
 9   مهلت بیمهٔ شخص ثالث    14264 non-null  object
 10  گیربکس                 14272 non-null  object
 11  نوع سوخت               13028 non-null  object
 12  قیمت فروش نقدی         15071 non-null  object
 13  وضعیت شاسی جلو         1088 non-null   object
 14  وضعیت شاسی عقب         1044 non-null   object
 15  نمایشگاه           

In [5]:
df['token'].nunique()

14358

In [6]:
df['دور رنگ'] = df['description'].str.contains('دور رنگ').astype(int)

In [7]:
df['کارکرد'] = df['کارکرد'].apply(lambda x: unidecode(x)).str.replace(',', '').astype(int)

In [8]:
df = df[df['مدل (سال تولید)'] != 'قبل از ۱۳۶۶']

In [9]:
df['مدل (سال تولید)'] = df['مدل (سال تولید)'].apply(lambda x: unidecode(x)).str.replace(',', '').astype(int)
df['مدل (سال تولید)'] = df['مدل (سال تولید)'].apply(lambda x: x - 622 if x >= 1900 else x)

In [10]:
df['مهلت بیمهٔ شخص ثالث'] = df['مهلت بیمهٔ شخص ثالث'].str.replace('ماه', '').apply(lambda x: unidecode(str(x))).astype(float)

In [11]:
df['قیمت فروش نقدی'] = df['قیمت فروش نقدی'].str.replace('تومان', '').apply(lambda x: unidecode(x)).str.replace(',', '').str.extract('(\d+)').astype(float)

In [12]:
df.head()

,token,description,کارکرد,مدل (سال تولید),رنگ,برند و تیپ,وضعیت موتور,وضعیت شاسی‌ها,وضعیت بدنه,مهلت بیمهٔ شخص ثالث,...,قیمت فروش نقدی,وضعیت شاسی جلو,وضعیت شاسی عقب,نمایشگاه,حداقل مبلغ پیش‌پرداخت,نحوهٔ فروش,تعداد اقساط,مبلغ قسط,فروشنده,دور رنگ
0,wYtmuTDf,ساعات بازدید روزهای عادی 17.30 به بعد.,242000,1385,مشکی,پراید، صندوق‌دار، بنزینی,سالم,سالم و پلمپ,خط و خش جزیی,1.0,...,93000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,wYO2A7PE,خودرو از لحاظ فنی بشرط کارشناسی بدنه چند لکه ...,200,1392,نقره‌ای,بایک، سابرینا، هاچبک,سالم,سالم و پلمپ,خط و خش جزیی,10.0,...,325000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,wYti-uhK,با سلام\nفنی سالم.بی رنگ.معاوضه با خودرو ارزان...,89000,1394,سفید,آریو، اتوماتیک 1600cc,سالم,سالم و پلمپ,سالم و بی‌خط و خش,6.0,...,430000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,wYteuShG,تمام سرویس ها تازه انجام شده ماشین کم کار و بس...,48000,1397,سفید,دانگ فنگ، H30 کراس,سالم,NaN,رنگ‌شدگی در ۱ ناحیه,12.0,...,465000000.0,سالم و پلمپ,ضربه‌خورده,NaN,NaN,NaN,NaN,NaN,NaN,0
4,wYtW-Q6S,تاکسی بین شهری slx بسیار تروتمیز موتور سالم گی...,60000,1389,زرد,پژو، 405، SLX بنزینی,سالم,سالم و پلمپ,دوررنگ,10.0,...,160000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [13]:
data = df.copy().drop(['token', 'description', 'نوع سوخت', 'وضعیت شاسی جلو', 'وضعیت شاسی عقب',\
     'نمایشگاه', 'حداقل مبلغ پیش‌پرداخت', 'نحوهٔ فروش', 'تعداد اقساط', 'مبلغ قسط', 'فروشنده'], axis = 1)
data.head(2)

,کارکرد,مدل (سال تولید),رنگ,برند و تیپ,وضعیت موتور,وضعیت شاسی‌ها,وضعیت بدنه,مهلت بیمهٔ شخص ثالث,گیربکس,قیمت فروش نقدی,دور رنگ
0,242000,1385,مشکی,پراید، صندوق‌دار، بنزینی,سالم,سالم و پلمپ,خط و خش جزیی,1.0,دنده‌ای,93000000.0,0
1,200,1392,نقره‌ای,بایک، سابرینا، هاچبک,سالم,سالم و پلمپ,خط و خش جزیی,10.0,اتوماتیک,325000000.0,1


In [14]:
brands = data.groupby('برند و تیپ', as_index=False)['کارکرد'].count()
brands.columns = ['brand', 'count']
brands.sort_values(by='count', inplace=True)
brands = brands[brands['count'] > 5]

In [15]:
data = data[data['برند و تیپ'].isin(brands['brand'])]

In [16]:
clr = data.groupby('رنگ', as_index=False)['قیمت فروش نقدی'].count()
clr = clr[clr['قیمت فروش نقدی'] > data.shape[0] / 50]
colors = list(clr['رنگ'].values)
colors

['آبی',
 'خاکستری',
 'سفید',
 'سفید صدفی',
 'مشکی',
 'نقره\u200cای',
 'نوک\u200cمدادی']

In [17]:
data['رنگ'] = data['رنگ'].apply(lambda x: x if x in colors else 'سایر')

In [18]:
categorical_cols = []
for col in data.columns:
    if data[col].dtypes == 'object':
        categorical_cols.append(col)
print(categorical_cols)

['رنگ', 'برند و تیپ', 'وضعیت موتور', 'وضعیت شاسی\u200cها', 'وضعیت بدنه', 'گیربکس']


In [19]:
data = pd.concat([data, pd.get_dummies(data[categorical_cols], drop_first=True)], axis=1)
data.head()

,کارکرد,مدل (سال تولید),رنگ,برند و تیپ,وضعیت موتور,وضعیت شاسی‌ها,وضعیت بدنه,مهلت بیمهٔ شخص ثالث,گیربکس,قیمت فروش نقدی,...,وضعیت بدنه_رنگ‌شدگی، در ۲ ناحیه,وضعیت بدنه_سالم و بی‌خط و خش,وضعیت بدنه_صافکاری بی‌رنگ,وضعیت بدنه_صافکاری بی‌رنگ، در ۱ ناحیه,وضعیت بدنه_صافکاری بی‌رنگ، در ۲ ناحیه,وضعیت بدنه_صافکاری بی‌رنگ، در ۳ ناحیه,وضعیت بدنه_صافکاری بی‌رنگ، در ۴ ناحیه,وضعیت بدنه_صافکاری بی‌رنگ، در ۵ ناحیه,وضعیت بدنه_صافکاری بی‌رنگ، در ۶ ناحیه,گیربکس_دنده‌ای
0,242000,1385,مشکی,پراید، صندوق‌دار، بنزینی,سالم,سالم و پلمپ,خط و خش جزیی,1.0,دنده‌ای,93000000.0,...,0,0,0,0,0,0,0,0,0,1
2,89000,1394,سفید,آریو، اتوماتیک 1600cc,سالم,سالم و پلمپ,سالم و بی‌خط و خش,6.0,اتوماتیک,430000000.0,...,0,1,0,0,0,0,0,0,0,0
3,48000,1397,سفید,دانگ فنگ، H30 کراس,سالم,NaN,رنگ‌شدگی در ۱ ناحیه,12.0,اتوماتیک,465000000.0,...,0,0,0,0,0,0,0,0,0,0
4,60000,1389,سایر,پژو، 405، SLX بنزینی,سالم,سالم و پلمپ,دوررنگ,10.0,دنده‌ای,160000000.0,...,0,0,0,0,0,0,0,0,0,1
5,8000,1401,سفید,پراید، وانت 151، SE,سالم,سالم و پلمپ,سالم و بی‌خط و خش,8.0,دنده‌ای,239000000.0,...,0,1,0,0,0,0,0,0,0,1


In [20]:
for col in categorical_cols:
    print(col, df[col].nunique())

رنگ 36
برند و تیپ 361
وضعیت موتور 3
وضعیت شاسی‌ها 3
وضعیت بدنه 26
گیربکس 2


In [21]:
X = data.dropna().iloc[:, data.columns!='قیمت فروش نقدی']
y = data.dropna().iloc[:, data.columns=='قیمت فروش نقدی']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)#, stratify=X['برند و تیپ'])

X_train.drop(categorical_cols, axis=1, inplace=True)
X_test.drop(categorical_cols, axis=1, inplace=True)

In [23]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Linear Regression

In [24]:
regressor_lr = LinearRegression()
regressor_lr.fit(X_train, y_train)
pred_lr = regressor_lr.predict(X_test)

In [25]:
scores_lr = cross_validate(regressor_lr,\
 X_train, X_train, cv=5,\
  scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'))

In [26]:
df_scores_lr = pd.DataFrame(scores_lr)
df_scores_lr['test_neg_root_mean_squared_error'] = df_scores_lr['test_neg_root_mean_squared_error'] * -1
df_scores_lr['test_neg_mean_absolute_error'] = df_scores_lr['test_neg_mean_absolute_error'] * -1
df_scores_lr.round(2)

,fit_time,score_time,test_r2,test_neg_root_mean_squared_error,test_neg_mean_absolute_error
0,0.79,0.06,-3.820000e+00,0.00,0.00
1,0.57,0.06,-1.962510e+28,0.09,0.00
2,0.57,0.07,-2.150000e+00,0.00,0.00
3,0.53,0.05,-2.865822e+26,0.04,0.00
4,0.55,0.05,-2.627737e+28,0.12,0.01


In [27]:
print('MAE = ', metrics.mean_absolute_error(y_test, pred_lr))
print('MSE = ', metrics.mean_squared_error(y_test, pred_lr))
print('RMSE = ', np.sqrt(metrics.mean_squared_error(y_test, pred_lr)))
print('R2 = ', metrics.r2_score(y_test, pred_lr))

MAE =  20137597.694201406
MSE =  1100708630588265.2
RMSE =  33176929.191657647
R2 =  0.9328840731657161


## Ridge Regression

In [28]:
regressor_rg = Ridge()
regressor_rg.fit(X_train, y_train)
pred_rg = regressor_rg.predict(X_test)

In [29]:
scores_rg = cross_validate(regressor_rg,\
 X_train, y_train, cv=5,\
  scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'))

In [30]:
df_scores_rg = pd.DataFrame(scores_rg)
df_scores_rg['test_neg_root_mean_squared_error'] = df_scores_rg['test_neg_root_mean_squared_error'] * -1
df_scores_rg['test_neg_mean_absolute_error'] = df_scores_rg['test_neg_mean_absolute_error'] * -1
df_scores_rg.round(2)

,fit_time,score_time,test_r2,test_neg_root_mean_squared_error,test_neg_mean_absolute_error
0,0.08,0.01,0.94,31775603.39,19690510.65
1,0.06,0.01,0.92,34762885.99,20145050.42
2,0.06,0.01,0.93,33279917.79,19811813.68
3,0.13,0.01,0.93,33297343.39,19304724.10
4,0.11,0.01,0.93,33578719.11,20913540.10


In [31]:
print('MAE = ', metrics.mean_absolute_error(y_test, pred_rg))
print('MSE = ', metrics.mean_squared_error(y_test, pred_rg))
print('RMSE = ', np.sqrt(metrics.mean_squared_error(y_test, pred_rg)))
print('R2 = ', metrics.r2_score(y_test, pred_rg))

MAE =  20079134.3868153
MSE =  1071725706511013.1
RMSE =  32737222.034116045
R2 =  0.9346513127037334


## KNN

In [32]:
KNN = GridSearchCV(KNeighborsRegressor(), param_grid = {'n_neighbors': [2, 3, 4, 5, 6, 7, 8]}, cv = 5, n_jobs = -1)

KNN.fit(X_train, y_train)
KNN.best_params_

{'n_neighbors': 4}

In [33]:
scores_knn = cross_validate(KNN.best_estimator_,\
 X_train, y_train, cv=5,\
  scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'))

In [34]:
df_scores_knn = pd.DataFrame(scores_knn)
df_scores_knn['test_neg_root_mean_squared_error'] = df_scores_knn['test_neg_root_mean_squared_error'] * -1
df_scores_knn['test_neg_mean_absolute_error'] = df_scores_knn['test_neg_mean_absolute_error'] * -1
df_scores_knn.round(2)

,fit_time,score_time,test_r2,test_neg_root_mean_squared_error,test_neg_mean_absolute_error
0,0.03,0.31,0.86,46209354.49,29187191.56
1,0.02,0.14,0.87,45061353.59,28248697.65
2,0.03,0.17,0.88,43525741.64,27963849.70
3,0.03,0.18,0.86,46176760.53,29055635.86
4,0.03,0.18,0.86,46431685.28,30426781.38


In [35]:
pred_knn = KNN.best_estimator_.predict(X_test)

In [36]:
print('MAE = ', metrics.mean_absolute_error(y_test, pred_knn))
print('MSE = ', metrics.mean_squared_error(y_test, pred_knn))
print('RMSE = ', np.sqrt(metrics.mean_squared_error(y_test, pred_knn)))
print('R2 = ', metrics.r2_score(y_test, pred_knn))

MAE =  28130681.81404788
MSE =  1965994085138804.2
RMSE =  44339531.85520574
R2 =  0.8801231211348897


# SVR

In [37]:
sc2 = StandardScaler()
y_train_sc = sc2.fit_transform(pd.DataFrame(y_train))
y_test_sc = sc2.transform(pd.DataFrame(y_test))

In [38]:
regressor_sv = SVR()
regressor_sv.fit(X_train, y_train_sc.ravel())

SVR()

In [39]:
scores_sv = cross_validate(regressor_sv,\
 X_train, y_train_sc.ravel(), cv=5,\
  scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'))

In [40]:
df_scores_sv = pd.DataFrame(scores_sv)
xsv = sc2.inverse_transform(df_scores_sv['test_neg_root_mean_squared_error'].values.reshape(1, -1) * -1).reshape(-1, 1)
ysv = sc2.inverse_transform(df_scores_sv['test_neg_mean_absolute_error'].values.reshape(1, -1) * -1).reshape(-1, 1)
df_scores_sv['test_neg_root_mean_squared_error'] = xsv
df_scores_sv['test_neg_mean_absolute_error'] = ysv
df_scores_sv.round(2)

,fit_time,score_time,test_r2,test_neg_root_mean_squared_error,test_neg_mean_absolute_error
0,8.53,2.08,0.92,3.592584e+08,3.443147e+08
1,6.68,2.08,0.91,3.602549e+08,3.441635e+08
2,7.10,2.50,0.92,3.585378e+08,3.436963e+08
3,7.16,2.39,0.91,3.599905e+08,3.440183e+08
4,8.23,2.77,0.92,3.590026e+08,3.442659e+08


In [41]:
pred_sv = regressor_sv.predict(X_test)

In [42]:
print('MAE = ', metrics.mean_absolute_error(sc2.inverse_transform(y_test_sc), sc2.inverse_transform(pred_sv.reshape(-1, 1))))
print('MSE = ', metrics.mean_squared_error(sc2.inverse_transform(y_test_sc), sc2.inverse_transform(pred_sv.reshape(-1, 1))))
print('RMSE = ', np.sqrt(metrics.mean_squared_error(sc2.inverse_transform(y_test_sc), sc2.inverse_transform(pred_sv.reshape(-1, 1)))))
print('R2 = ', metrics.r2_score(sc2.inverse_transform(y_test_sc), sc2.inverse_transform(pred_sv.reshape(-1, 1))))

MAE =  20960197.47325529
MSE =  1302789269408458.5
RMSE =  36094172.236089006
R2 =  0.9205621661752776


## Random Forest

In [43]:
random_grid = {
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [400, 600, 800, 1000]
}

In [44]:
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 5, cv = 5, verbose=1, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train.values.ravel())

Fitting 5 folds for each of 5 candidates, totalling 25 fits


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=5, n_jobs=-1,
                   param_distributions={'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [400, 600, 800, 1000]},
                   random_state=42, verbose=1)

In [45]:
scores_rf = cross_validate(rf_random.best_estimator_,\
 X_train, y_train.values.ravel(), cv=5,\
  scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'))

In [46]:
df_scores_rf = pd.DataFrame(scores_rf)
df_scores_rf['test_neg_root_mean_squared_error'] = df_scores_rf['test_neg_root_mean_squared_error'] * -1
df_scores_rf['test_neg_mean_absolute_error'] = df_scores_rf['test_neg_mean_absolute_error'] * -1
df_scores_rf.round(2)

,fit_time,score_time,test_r2,test_neg_root_mean_squared_error,test_neg_mean_absolute_error
0,56.58,0.31,0.90,39604451.89,24476003.79
1,42.20,0.29,0.90,40278362.34,23871936.84
2,43.90,0.29,0.89,40530412.78,24853259.39
3,45.68,0.29,0.89,40921466.79,24911183.28
4,45.28,0.30,0.90,40279348.89,24709456.68


In [47]:
pred_rf = rf_random.best_estimator_.predict(X_test)

In [48]:
print('MAE = ', metrics.mean_absolute_error(y_test, pred_rf))
print('MSE = ', metrics.mean_squared_error(y_test, pred_rf))
print('RMSE = ', np.sqrt(metrics.mean_squared_error(y_test, pred_rf)))
print('R2 = ', metrics.r2_score(y_test, pred_rf))

MAE =  24344462.85024015
MSE =  1613727726217638.8
RMSE =  40171230.08096265
R2 =  0.9016026321648858


# XGBoost

In [49]:
params = { 
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.03, 0.07, 0.1],
    'n_estimators': [100, 500, 800, 1000]
}

In [50]:
xg = XGBRegressor()
xg_random = RandomizedSearchCV(estimator = xg, param_distributions = params, n_iter = 5, cv = 5, verbose=1, random_state=42, n_jobs = -1)
xg_random.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, gpu_id=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                          max_delta_step=None, max_depth=None,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=100, n_jobs=None,
                                          num_parallel_tree=None,
                                          predictor=None, random_state=None, ...),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.03, 0.07,
                                                          0.1],
                                        'max_depth': [3, 6, 10],
                                        'n_estimators': [100, 500, 800, 1000]},
                   random_state=42, verbose=1)

In [51]:
scores_xg = cross_validate(xg_random.best_estimator_,\
 X_train, y_train, cv=5,\
  scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'))

In [52]:
df_scores_xg = pd.DataFrame(scores_xg)
df_scores_xg['test_neg_root_mean_squared_error'] = df_scores_xg['test_neg_root_mean_squared_error'] * -1
df_scores_xg['test_neg_mean_absolute_error'] = df_scores_xg['test_neg_mean_absolute_error'] * -1
df_scores_xg.round(2)

,fit_time,score_time,test_r2,test_neg_root_mean_squared_error,test_neg_mean_absolute_error
0,20.13,0.04,0.94,30794859.24,17880358.26
1,21.95,0.04,0.92,34474067.05,18851662.07
2,19.74,0.03,0.93,32663004.18,18483584.91
3,20.45,0.04,0.92,35481620.56,18892168.31
4,19.48,0.04,0.92,34735601.15,19497946.48


In [53]:
pred_xg = xg_random.best_estimator_.predict(X_test)

In [54]:
print('MAE = ', metrics.mean_absolute_error(y_test, pred_xg))
print('MSE = ', metrics.mean_squared_error(y_test, pred_xg))
print('RMSE = ', np.sqrt(metrics.mean_squared_error(y_test, pred_xg)))
print('R2 = ', metrics.r2_score(y_test, pred_xg))

MAE =  18730051.318932306
MSE =  1043636442494482.8
RMSE =  32305362.441775557
R2 =  0.9363640611424867


In [55]:
df['قیمت فروش نقدی'].mean()

314542442.9442117

## Feature Importances

In [56]:
importances = pd.DataFrame(zip(X.columns, xg_random.best_estimator_.feature_importances_), columns=['feature', 'importance'])

In [57]:
importances['root_feature'] = importances['feature'].apply(lambda x: x.split('_')[0])

In [58]:
root_importances = importances.groupby('root_feature', as_index=False)['importance'].sum().sort_values(by='importance', ascending=False)

In [59]:
root_importances

,root_feature,importance
0,برند و تیپ,0.920884
5,وضعیت بدنه,0.042694
2,رنگ,0.019499
3,مدل (سال تولید),0.012927
8,کارکرد,0.001323
7,وضعیت موتور,0.001302
6,وضعیت شاسی‌ها,0.000764
9,گیربکس,0.000451
1,دور رنگ,0.000079
4,مهلت بیمهٔ شخص ثالث,0.000078


# Evaluation

In [60]:
res = pd.DataFrame([[metrics.mean_absolute_error(y_test, pred_i) for pred_i in [pred_rg, pred_knn, pred_rf, pred_xg]] + [metrics.mean_absolute_error(sc2.inverse_transform(y_test_sc), sc2.inverse_transform(pred_sv.reshape(-1, 1)))],\
    [np.sqrt(metrics.mean_squared_error(y_test, pred_i)) for pred_i in [pred_rg, pred_knn, pred_rf, pred_xg]] + [np.sqrt(metrics.mean_squared_error(sc2.inverse_transform(y_test_sc), sc2.inverse_transform(pred_sv.reshape(-1, 1))))],\
        [metrics.r2_score(y_test, pred_i) for pred_i in [pred_rg, pred_knn, pred_rf, pred_xg]] + [metrics.r2_score(sc2.inverse_transform(y_test_sc), sc2.inverse_transform(pred_sv.reshape(-1, 1)))]]).transpose()

np.sqrt(metrics.mean_squared_error(sc2.inverse_transform(y_test_sc), sc2.inverse_transform(pred_sv.reshape(-1, 1))))

res.index = ['Ridge regression', 'KNN', 'Random Forest', 'Xgboost', 'SVR']
res.columns = ['MAE', 'RMSE', 'R2']
res.sort_values(by='MAE')

,MAE,RMSE,R2
Xgboost,1.873005e+07,3.230536e+07,0.936364
Ridge regression,2.007913e+07,3.273722e+07,0.934651
SVR,2.096020e+07,3.609417e+07,0.920562
Random Forest,2.434446e+07,4.017123e+07,0.901603
KNN,2.813068e+07,4.433953e+07,0.880123
